```
--- Day 14: Extended Polymerization ---
The incredible pressures at this depth are starting to put a strain on your submarine. The submarine has polymerization equipment that would produce suitable materials to reinforce the submarine, and the nearby volcanically-active caves should even have the necessary input elements in sufficient quantities.

The submarine manual contains instructions for finding the optimal polymer formula; specifically, it offers a polymer template and a list of pair insertion rules (your puzzle input). You just need to work out what polymer would result after repeating the pair insertion process a few times.

For example:

NNCB

CH -> B
HH -> N
CB -> H
NH -> C
HB -> C
HC -> B
HN -> C
NN -> C
BH -> H
NC -> B
NB -> B
BN -> B
BB -> N
BC -> B
CC -> N
CN -> C
The first line is the polymer template - this is the starting point of the process.

The following section defines the pair insertion rules. A rule like AB -> C means that when elements A and B are immediately adjacent, element C should be inserted between them. These insertions all happen simultaneously.

So, starting with the polymer template NNCB, the first step simultaneously considers all three pairs:

The first pair (NN) matches the rule NN -> C, so element C is inserted between the first N and the second N.
The second pair (NC) matches the rule NC -> B, so element B is inserted between the N and the C.
The third pair (CB) matches the rule CB -> H, so element H is inserted between the C and the B.
Note that these pairs overlap: the second element of one pair is the first element of the next pair. Also, because all pairs are considered simultaneously, inserted elements are not considered to be part of a pair until the next step.

After the first step of this process, the polymer becomes NCNBCHB.

Here are the results of a few steps using the above rules:

Template:     NNCB
After step 1: NCNBCHB
After step 2: NBCCNBBBCBHCB
After step 3: NBBBCNCCNBBNBNBBCHBHHBCHB
After step 4: NBBNBNBBCCNBCNCCNBBNBBNBBBNBBNBBCBHCBHHNHCBBCBHCB
This polymer grows quickly. After step 5, it has length 97; After step 10, it has length 3073. After step 10, B occurs 1749 times, C occurs 298 times, H occurs 161 times, and N occurs 865 times; taking the quantity of the most common element (B, 1749) and subtracting the quantity of the least common element (H, 161) produces 1749 - 161 = 1588.

Apply 10 steps of pair insertion to the polymer template and find the most and least common elements in the result. What do you get if you take the quantity of the most common element and subtract the quantity of the least common element?
```

In [1]:
import numpy as np

In [2]:
def get_inputs(infile):
    """
    Load this weeks example/input files
    
    parameters
    ----------
    infile : str
        The name of the input file
    
    returns
    -------
    template, rules : str, dict
        Template is the starting polymer
        rules = { AB:C, ...} for the rule "AB -> C" etc 
    """
    template, rules = open(infile).read().split('\n\n')
    template = template.strip()
    rules = rules.split('\n')[:-1]
    rules = dict([ a.split(' -> ') for a in rules])
    return template, rules

In [3]:
def next_step(template, rules):
    """
    Perform one iteration of the polymerization process
    
    parameters
    ----------
    template : str
        starting polymer
        
    rules: dict
        rules = { AB:C, ...} for the rule "AB -> C" etc
      
    returns
    -------
    template : str
        The evolved / modified templates
    """
    t = template
    nt = ''
    for a,b in zip(template[:-1], template[1:]):
        nt += a
        nt += rules.get( f"{a}{b}", '')
    nt += t[-1]
    return nt

In [4]:
def do_steps(template, rules, nsteps):
    """
    Iterate over multiple steps of the polymerization process
    
    parameters
    ----------
    template : str
        starting polymer
        
    rules: dict
        rules = { AB:C, ...} for the rule "AB -> C" etc
    
    nsteps : int
        The number of steps to perform
      
    returns
    -------
    template : str
        The evolved / modified templates
    """
    t = template
    for i in range(nsteps):
        t = next_step(t,rules)
    return t
        

In [5]:
def count_elements(polymer):
    """
    Count the number of occurances of each element
    Report the difference between the maximum and minimum
    
    parameters
    ----------
    polymer : str
        The starting polymer template
        
    returns
    -------
    count : float
        The max count minus the min count
    """
    elements = frozenset(polymer)
    counts = np.zeros(len(elements), dtype=int)
    for i,s in enumerate(elements):
        counts[i] = polymer.count(s)
    mx = np.max(counts)
    mn = np.min(counts)
    return mx-mn

In [6]:
def solve1(infile):
    template, rules = get_inputs(infile)
    ten_steps = do_steps(template,rules, 10)
    answer = count_elements(ten_steps)
    print(f"The max-min count is {answer}")

In [7]:
solve1('ex1.txt')

The max-min count is 1588


In [8]:
solve1('input.txt')

The max-min count is 3048


```
--- Part Two ---
The resulting polymer isn't nearly strong enough to reinforce the submarine. You'll need to run more steps of the pair insertion process; a total of 40 steps should do it.

In the above example, the most common element is B (occurring 2192039569602 times) and the least common element is H (occurring 3849876073 times); subtracting these produces 2188189693529.

Apply 40 steps of pair insertion to the polymer template and find the most and least common elements in the result. What do you get if you take the quantity of the most common element and subtract the quantity of the least common element?

```

In [9]:
import itertools

In [10]:
def get_elements(rules):
    """
    Determine the set of all possible elements by inspecting a rule set
    
    parameters
    ----------
    rules : dict
        rules = { AB:C, ...} for the rule "AB -> C" etc
        
    returns
    -------
    elements : frozenset
        A frozenset of all the possible (single) elements in the rule set
    """
    elements = frozenset(list(rules.values()))
    return elements

In [11]:
def decompose_pairs(polymer, elements):
    """
    Turn a string into a dict of all the consecutive pairs of characters.
    The pairs are the keys, and the count of occurances are the values.
    
    parameters
    ----------
    polymer : str
        The input polymer
        
    elements : frozenset
        The possible elements in our rule set
        
    returns
    -------
    pairs, empty : dict, dict
        All possible pairs of elements. pairs contains the counts, empty has all counts=0
    """
    pairs = dict( (''.join(p),0) for p in itertools.product(''.join(elements), repeat=2))
    empty= pairs.copy()
    
    for a,b in zip(polymer[:-1], polymer[1:]):
        pairs[f"{a}{b}"] +=1
        
    return pairs, empty

In [12]:
def count_element2(pairs, elements, template):
    """
    Count the number of occurances of each element
    Report the difference between the maximum and minimum
    
    parameters
    ----------
    pairs : dict
        pairs = {"AB":count, ... } for each possible pairing of elements
    
    elements : frozenset
        all the possible single elements
        
    template : str
        The starting polymer template
        
    returns
    -------
    count : float
        The max count minus the min count
    """
    counts = dict( (e,0) for e in elements)
    # all elements are double counted except the first and last so we correct this
    counts[template[0]] = 1
    counts[template[-1]] = 1
    for k in pairs:
        for s in k:
            counts[s] += pairs[k]
    # all are double counted so the min/max is just 1/2 the value
    mx = np.max(list(counts.values()))/2
    mn = np.min(list(counts.values()))/2
    return mx-mn
    

In [13]:
def solve2(infile):
    template, rules = get_inputs(infile)
    elements = get_elements(rules)

    pairs, empty = decompose_pairs(template, elements)
    
    for _ in range(40):
        c_pairs = empty.copy()
        for p in pairs:
            # if AB->C then pairs[AB] -=1, and pairs[AC]+=1, pairs[CB]+=1
            out = rules[p]
            k1 = p[0]+out
            k2 = out+p[1]
            n = pairs[p]
            if n>0:
                c_pairs[k1] += n
                c_pairs[k2] += n
                c_pairs[p]  -= n
        for p in c_pairs:
            pairs[p] += c_pairs[p]
    
    answer = int(count_element2(pairs, elements, template))
    print(f"The max-min count is {answer}")

In [14]:
solve2('ex1.txt')

The max-min count is 2188189693529


In [15]:
solve2('input.txt')

The max-min count is 3288891573057
